In [6]:
import pandas as pd
import numpy as np
from random import *
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score

import my_pickle as mp
import my_features as mf
import my_resample as ms
# from my_resample import div_count_pos_neg, undersample, oversample
from sklearn.model_selection import GridSearchCV


from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_curve, auc
from scipy import interp

In [7]:
X = mp.unpickle_it('data_features')
print("Make sure we're not useing ECT data")
print(pd.to_datetime(X.timestamp.max()*1000000))
X.head()

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


,conv_id,response,first_uid,second_uid,first_mid,second_mid,timestamp,created_sender,updated_sender,activeAt_sender,...,same_relate,same_clean,same_night,same_student,same_smoking,same_term,roommate_similarity,hobbies_overlap,amenities_overlap,rent_overlap
16296,rKdn0zblFiRVDUEmDHyQ,True,rKdn0zblFi,RVDUEmDHyQ,-KWKghvlpF67UOv-iXXO,-KWKg_ybWV4qbG02NYrb,1478906535730,2016-11-11 23:20:33.099,2017-02-02 01:38:06.265,NaT,...,0,0,1,0,0,0,2,1,0,175.0
9241,aXnWmLjpaERVDUEmDHyQ,False,aXnWmLjpaE,RVDUEmDHyQ,-KWLsSEmEJgH3bhJupE7,None,1478926390334,2016-11-11 23:59:37.980,2017-02-02 01:37:54.922,NaT,...,1,0,1,0,0,0,1,0,0,175.0
1069,8Ws3QWmC4B52lmw7KQcq,True,8Ws3QWmC4B,52lmw7KQcq,-KWPb8hd_DbLpY1eIUlT,-KWQNe2ghZCGV0aywnKp,1478988962771,2016-11-12 14:16:18.686,2018-01-12 22:04:11.499,2017-09-26 20:12:39.006,...,1,1,1,0,1,0,0,2,3,1000.0
9414,b1HYtkC11e92RR2gpmjr,False,b1HYtkC11e,92RR2gpmjr,-KWQMn5WmLIKbUOt9a0O,None,1479001715202,2016-11-12 19:36:36.835,2018-01-12 09:23:09.942,2017-10-04 15:41:18.274,...,1,1,0,0,1,1,2,1,2,0.0
4142,KnmYDsn5Q492RR2gpmjr,False,KnmYDsn5Q4,92RR2gpmjr,-KWQNFIghvLurnI-8epo,None,1479001834795,2016-11-13 01:49:12.305,2017-03-14 18:32:05.164,NaT,...,1,0,0,0,1,0,1,1,0,175.0


In [15]:
X.iloc[:,30:40].head()

,term_sender,type_sender,has_room_sender,amenities_sender,hobbies_sender,inRelationship_sender,isClean_sender,isNight_sender,isStudent_sender,petsOk_sender
16296,12.0,private,False,{Dishwasher},"{Movies, Sports}",0.0,0.0,0.0,NaN,0.0
9241,12.0,private,False,"{Dishwasher, Furnished}","{Running, Yoga}",1.0,0.0,0.0,NaN,0.0
1069,10.0,private,False,"{Dishwasher, Furnished, Parking}","{TV Shows, Sports, Socializing, Running, Gaming}",1.0,0.0,1.0,0.0,1.0
9414,12.0,private,False,"{Furnished, Private Bath}","{Gaming, Socializing, Art}",1.0,0.0,0.0,0.0,1.0
4142,NaN,NaN,True,{},"{Book Worm, Running}",1.0,1.0,0.0,NaN,0.0


In [38]:
X = mp.unpickle_it('data_features')
y = X.response.map({True:1,False:0})

cols_to_keep = [
     'has_about_sender','len_about_sender', 'age_sender', 'gender_sender',
#        'facebookId_sender', 'linkedinId_sender', 'picture_sender',
#        'maxCost_sender', 'minCost_sender', 
#        'numRoommates_sender', 'term_sender', 'type_sender', 'has_room_sender',
#         'inRelationship_sender','isClean_sender', 'isNight_sender', 'isStudent_sender', 
            'petsOk_sender','smokingOk_sender',

                
       
#        'has_about_receiver', 'len_about_receiver', 'age_receiver', 'gender_receiver', 
#        'facebookId_receiver', 'linkedinId_receiver', 'picture_receiver', 
#         'maxCost_receiver', 'minCost_receiver', 
#         'numRoommates_receiver', 'term_receiver', 'type_receiver', 'has_room_receiver',
#         'inRelationship_receiver', 'isClean_receiver', 'isNight_receiver', 'isStudent_receiver',
       'petsOk_receiver', 'smokingOk_receiver',
        
        'age_dif',
       'same_gender', 'same_relate', 'same_clean', 'same_night',
       'same_student', 'same_smoking', 'same_term', 'roommate_similarity']
X = X[cols_to_keep]
X.head()

,has_about_sender,len_about_sender,age_sender,gender_sender,petsOk_sender,smokingOk_sender,petsOk_receiver,smokingOk_receiver,age_dif,same_gender,same_relate,same_clean,same_night,same_student,same_smoking,same_term,roommate_similarity
16296,False,0,29.0,female,0.0,0.0,1.0,1.0,11.0,1,0,0,1,0,0,0,2
9241,False,0,29.0,male,0.0,0.0,1.0,1.0,11.0,0,1,0,1,0,0,0,1
1069,False,0,26.0,male,1.0,0.0,1.0,0.0,2.0,1,1,1,1,0,1,0,0
9414,True,82,29.0,male,1.0,0.0,1.0,0.0,1.0,1,1,1,0,0,1,1,2
4142,False,0,29.0,female,0.0,0.0,1.0,0.0,1.0,0,1,0,0,0,1,0,1


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

X_train, y_train = ms.oversample(X_train, y_train, .5)

# print("messages sent: {}".format(len(y_train)))
# print("responses:     {}\n".format(y_train.sum()))

model = RandomForestClassifier(max_depth= 100, max_features=None, min_samples_leaf=3, n_estimators= 100, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\nMETRICS")
# print("Prediction:   {}...".format(model.predict(X_test)[:20]))
# print("Actual:       {}...".format(y_test[:20]))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("Model accuracy: {}".format(model.score(X_test, y_test)))

pred_all_0 = [0]*len(y_test)
pred_all_1 = [1]*len(y_test)
pred_50_50 = np.random.choice([0,1], size=len(y_test))
pred_90_10 = np.random.choice([0,1], size=len(y_test), p=[.9,.1])

print ("\nCONFUSION MATRIX")
print (confusion_matrix(y_test, y_pred))
print ("\nkey:")
print (" TN   FP ")
print (" FN   TP ")

# print(model.feature_importances_)

feature_importances = np.argsort(model.feature_importances_)
top_n = len(X.columns)
print("\nFEATURE RANKINGS")
for n in range(top_n):
    print(n+1, '\t',X.columns[feature_importances[-n-1]], '\t',sorted(model.feature_importances_)[-n-1])
       
# TO DOUBLE CHECK STUFF:    
# for n in range(top_n):   
#     print(model.feature_importances_[n], X.columns[n])

print("\nRECALL AND ACCURACY FOR DIFFERNET MODELS")
print("recall     \t precision   \tmodel")
print(recall_score(y_test, y_pred), '\t',precision_score(y_test, y_pred), "my model")
print(recall_score(y_test, pred_all_0),'\t','\t', precision_score(y_test, pred_all_0), "\t\tpredict all zero")
print(recall_score(y_test, pred_all_1),'\t','\t', precision_score(y_test, pred_all_1), "predict all one")
print(recall_score(y_test, pred_50_50),'\t', precision_score(y_test, pred_50_50), "predict 50-50")
print(recall_score(y_test, pred_90_10), precision_score(y_test, pred_90_10), "predict 90-10")

ValueError: could not convert string to float: 'male'

In [ ]:
# # Calculate the standard deviation for feature importances across all trees

# n = 10 # top 10 features

# #importances = forest_fit.feature_importances_[:n]
# importances = model.feature_importances_[:n]
# std = np.std([tree.feature_importances_ for tree in model.estimators_],
#              axis=0)
# indices = np.argsort(importances)[::-1]
# features = list(X.columns[indices])

# print("Feature ranking:")

# for f in range(5):
#     print("%d. %s (%f)" % (f + 1, features[f], importances[indices[f]]))


In [ ]:
# Calculate the standard deviation for feature importances across all trees

n = len(X.columns)

#importances = forest_fit.feature_importances_[:n]
importances = model.feature_importances_[:n]
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
features = list(X.columns[indices])

print("Feature ranking:")

for f in range(n):
    print("%d. %s (%f)" % (f + 1, features[f], importances[indices[f]]))


In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(n), importances[indices], yerr=std[indices], color="r", align="edge", width = -.9)
plt.xticks(range(n), features, rotation=-75)
plt.xlim([-1, 5])
plt.show()

In [ ]:
# Try modifying the number of trees, graph results

num_trees = range(5, 50, 5)
accuracies = []
for n in num_trees:
    tot = 0
    for i in range(5):
        rf = RandomForestClassifier(n_estimators=n)
        rf.fit(X_train, y_train)
        tot += rf.score(X_test, y_test)
    accuracies.append(tot / 5)
plt.plot(num_trees, accuracies)
plt.xlabel="num_trees"
plt.ylabel="accuracy"
plt.show()

In [ ]:
# Modifying the max features parameter
for nn in range(10):
    num_features = range(2, len(X.columns))
    accuracies = []
    for n in num_features:
        tot = 0
        for i in range(5):
            rf = RandomForestClassifier(max_features=n)
            rf.fit(X_train, y_train)
            tot += rf.score(X_test, y_test)
        accuracies.append(tot / 5)
    plt.plot(num_features, accuracies)
    plt.xlabel="num_features"
    plt.ylabel="accuracy"
plt.show()

In [ ]:
# Run all the other classifiers that we have learned so far in class
def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
    model = classifier(**kwargs)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return model.score(X_test, y_test), \
           precision_score(y_test, y_predict), \
           recall_score(y_test, y_predict)

print ("    Model,                Accuracy, Precision, Recall")
print ("    Random Forest:       ", get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test, n_estimators=25, max_features=5))
print ("    Logistic Regression: ", get_scores(LogisticRegression, X_train, X_test, y_train, y_test))
print ("    Decision Tree:       ", get_scores(DecisionTreeClassifier, X_train, X_test, y_train, y_test))
print ("    Naive Bayes:         ", get_scores(MultinomialNB, X_train, X_test, y_train, y_test))

In [ ]:
def plot_roc(X, y, clf_class, title, **kwargs):
# def plot_roc(X, y, clf_class, kwargs):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_prob = np.zeros((len(y),2))
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []
    for i, (train_index, test_index) in enumerate(kf):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        # Predict probabilities, not classes
        y_prob[test_index] = clf.predict_proba(X_test)
        fpr, tpr, thresholds = roc_curve(y[test_index], y_prob[test_index, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    mean_tpr /= len(kf)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--',label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
    plt.title(title + 'ROC')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
xfake, yfake = np.random.rand(100,6), np.random.randint(0,2, size=(100,))
plt.figure(figsize=(20,16))
print ("Visualize the roc curve of each model")
plot_roc(xfake, yfake, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
#plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
#plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')

In [ ]:
print ("Visualize the roc curve of each model")
plot_roc(X, y, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')

In [ ]:
# def do_grid_search(X, y):
#     '''
#     X as 2d numpy array
#     y as 1d numpy array
    
#     PARAMETERS
#     n_estimators: The number of trees in the forest
#     criterion: gini or entropy
#     max_features: The number of features to consider when looking for the best split
#         If int, then consider max_features features at each split.
#         If float, then max_features is a percentage and int(max_features * n_features) features are considered at each split.
#         If “auto”, then max_features=sqrt(n_features).
#         If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).
#         If “log2”, then max_features=log2(n_features).
#         If None, then max_features=n_features.
#     max_depth: The maximum depth of the tree
#     n_jobs: The number of jobs to run in parallel for both fit and predict. If -1, then the number of jobs is set to the number of cores.
#     '''
    
#     # Split it up into our training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y)

#     # resample
#     X_train, y_train = ms.oversample(X_train, y_train, .5)
    
#     # Initalize our model here
#     model = RandomForestClassifier()

#     # Here are the params we are tuning
#     param_grid = {'max_features' : ["sqrt","log2",None],
#                   'n_estimators' : [100],
#                   'max_depth': [10, 50, 100, 200],
#                   'min_samples_leaf': [3,5,10]
#                   }

#     # Plug in our model, params dict, and the number of jobs, then .fit()
#     gs_cv = GridSearchCV(model, param_grid, n_jobs=-1).fit(X_train, y_train)

#     # return the best score and the best params
#     return gs_cv.best_score_, gs_cv.best_params_
